In [25]:
from datasets import load_dataset

raw_datasets = load_dataset(
    "json", 
    data_files={
        "train": "archive/train.json", 
        "validation": "archive/dev.json", 
        "test": "archive/test.json"
    }
)


On fait marcher le modèle de Greg

In [32]:
import sys
sys.path.append("graph")

from graph.predict import predict

In [36]:
probas = predict(list(raw_datasets["test"]), model_type="rgcn")
prob1 = probas[:, 1]

Using local data: C:\Users\octav\OneDrive\Bureau\hackathon\HackEurope-2026-Hoga\archive
Loading dataset splits...
  Loading train.json...
  Loading dev.json...
  Loading test.json...
  Labeled users: 11826 (train=8278, val=2365, test=1183)
  Total nodes: 191582 (11826 labeled, 179756 unlabeled neighbors)
Building edge index...
  Edges: 208716 (following=105701, follower=103015)
Extracting node features...
  Feature matrix: torch.Size([191582, 20])
  Labels: 5237 human, 6589 bot, 179756 unlabeled
  Masks: train=8278, val=2365, test=1183


On fait marcher le modèle d'Hadrien

On fait marcher le modèle de Aziz

In [48]:
from aziz_code import create_csv, create_features3, predict_bot_probability

In [49]:
import pickle
import pandas as pd
# ── Charger le modèle ─────────────────────────────────────────────────────────
with open('bot_detector.pkl', 'rb') as f:
    saved = pickle.load(f)
model    = saved['model']
features = saved['features']

In [47]:
df = create_csv(raw_datasets["test"])
df = create_features3(df)

results = predict_bot_probability(df, model,features)

TypeError: create_csv() missing 2 required positional arguments: 'dev_data' and 'test_data'

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

on fait les trucs qui contiendront les différentes méthodes

On a df_prob qui a prob1, prob2, prob3 et label

In [ ]:
acc1 = accuracy_score(df_prob["label"], (df_prob["prob1"] >= 0.5).astype(int))
acc2 = accuracy_score(df_prob["label"], (df_prob["prob2"] >= 0.5).astype(int))
acc3 = accuracy_score(df_prob["label"], (df_prob["prob3"] >= 0.5).astype(int))

print(f"Acc modèle 1: {acc1:.4f}")
print(f"Acc modèle 2: {acc2:.4f}")
print(f"Acc modèle 3: {acc3:.4f}")

In [ ]:
df_resultprob = pd.DataFrame()
df_pred = pd.DataFrame()

In [ ]:
#Moyenne normale des probas
df_resultprob["avg"] = df_prob[["prob1", "prob2", "prob3"]].mean(axis=1)
df_pred["avg"] = (df_resultprob["avg"] >= 0.5).astype(int)
print(f"Acc avg: {accuracy_score(df_prob['label'], df_pred['avg']):.4f}")

In [ ]:
#Moyenne pondérée par les accuracy
total = acc1 + acc2 + acc3
df_resultprob["wavg"] = (acc1 * df_prob["prob1"] + acc2 * df_prob["prob2"] + acc3 * df_prob["prob3"]) / total
df_pred["wavg"] = (df_resultprob["wavg"] >= 0.5).astype(int)
print(f"Acc wavg: {accuracy_score(df_prob['label'], df_pred['wavg']):.4f}")

In [ ]:
# ElasticNet
from sklearn.linear_model import ElasticNet

lam = 0.01
l1_ratio = 0.5

enet = ElasticNet(alpha=lam, l1_ratio=l1_ratio)
enet.fit(df_prob[["prob1", "prob2", "prob3"]], df_prob["label"])
df_resultprob["enet"] = enet.predict(df_prob[["prob1", "prob2", "prob3"]]).clip(0, 1)
df_pred["enet"] = (df_resultprob["enet"] >= 0.5).astype(int)
print(f"Acc ElasticNet (lambda={lam}, l1_ratio={l1_ratio}): {accuracy_score(df_prob['label'], df_pred['enet']):.4f}")

In [ ]:
# Régression logistique
from sklearn.linear_model import LogisticRegression

n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

df_resultprob["logreg"] = np.nan
for train_idx, val_idx in kf.split(df_prob):
    logreg = LogisticRegression()
    logreg.fit(df_prob.loc[train_idx, ["prob1", "prob2", "prob3"]], df_prob.loc[train_idx, "label"])
    df_resultprob.loc[val_idx, "logreg"] = logreg.predict_proba(df_prob.loc[val_idx, ["prob1", "prob2", "prob3"]])[:, 1]

df_pred["logreg"] = (df_resultprob["logreg"] >= 0.5).astype(int)
print(f"Acc LogReg (KFold): {accuracy_score(df_prob['label'], df_pred['logreg']):.4f}")

In [ ]:
# LightGBM
import lightgbm as lgb

df_resultprob["lgbm"] = np.nan
for train_idx, val_idx in kf.split(df_prob):
    lgb_model = lgb.LGBMClassifier(n_estimators=100, max_depth=3, learning_rate=0.1, verbose=-1)
    lgb_model.fit(df_prob.loc[train_idx, ["prob1", "prob2", "prob3"]], df_prob.loc[train_idx, "label"])
    df_resultprob.loc[val_idx, "lgbm"] = lgb_model.predict_proba(df_prob.loc[val_idx, ["prob1", "prob2", "prob3"]])[:, 1]

df_pred["lgbm"] = (df_resultprob["lgbm"] >= 0.5).astype(int)
print(f"Acc LightGBM (KFold): {accuracy_score(df_prob['label'], df_pred['lgbm']):.4f}")